In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import re
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
from requests import get
from bs4 import BeautifulSoup
from scipy import stats
import warnings
warnings.filterwarnings('ignore')


#import geopandas as gpd

from mpl_toolkits.basemap import Basemap
from IPython.display import set_matplotlib_formats
import os
import os.path
from geopy.geocoders import Nominatim

from matplotlib.collections import LineCollection


from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import matplotlib.cm as cm
import matplotlib.colors as colors
from matplotlib.colors import Normalize

In [3]:
inspections = pd.read_csv('food-inspections.csv')


In [8]:
# Remove trailing backslash (left over from sodapy conversion of "License #" column)
inspections.rename(columns={"License #": "License"}, inplace=True)

# Drop rows with missing data
inspections.dropna(subset=["Inspection Date", "License", "Latitude", "Longitude"], inplace=True)

# Drop duplicates
inspections.drop_duplicates("Inspection ID", inplace=True)

# Drop "0" licenses
inspections = inspections[inspections.License != "0"]

# Only consider successful inspections
inspections = inspections[~inspections.Results.isin(["Out of Business", "Business Not Located", "No Entry"])]

In [10]:
# Convert latitude & longitude to floats
inspections.Latitude = inspections.Latitude.astype(float)
inspections.Longitude = inspections.Longitude.astype(float)

In [12]:
# Only consider canvass inspections (not complaints or re-inspections)
inspections = inspections[inspections['Inspection Type'] == "Canvass"]

In [13]:
# Only consider restaurants and grocery stores (subject to change)
inspections = inspections[inspections['Facility Type'].isin(["Restaurant", "Grocery Store"])]

In [17]:
inspections

,Inspection ID,DBA Name,AKA Name,License,Facility Type,Risk,Address,City,State,Zip,...,Results,Violations,Latitude,Longitude,Location,Historical Wards 2003-2015,Zip Codes,Community Areas,Census Tracts,Wards
1,2320342,YOLK TEST KITCHEN,YOLK TEST KITCHEN,2589655.0,Restaurant,Risk 1 (High),1767 N MILWAUKEE AVE,CHICAGO,IL,60647.0,...,Pass w/ Conditions,23. PROPER DATE MARKING AND DISPOSITION - Comm...,41.913588,-87.682203,"{'longitude': '41.9135877900482', 'latitude': ...",NaN,NaN,NaN,NaN,NaN
5,2320261,CRAZY BIRD,CRAZY BIRD,2616198.0,Restaurant,Risk 1 (High),1160 W GRAND AVE,CHICAGO,IL,60642.0,...,Pass w/ Conditions,16. FOOD-CONTACT SURFACES: CLEANED & SANITIZED...,41.891193,-87.657055,"{'longitude': '41.891193484441246', 'latitude'...",NaN,NaN,NaN,NaN,NaN
7,2320298,HAISOUS,CA PHE DA,2464411.0,Restaurant,Risk 1 (High),1800-1802 S CARPENTER ST,CHICAGO,IL,60608.0,...,Pass,47. FOOD & NON-FOOD CONTACT SURFACES CLEANABLE...,41.857963,-87.653358,"{'longitude': '41.8579631274509', 'latitude': ...",NaN,NaN,NaN,NaN,NaN
10,2320229,HIMALAYAN SHERPA KITCHEN,HIMALAYAN SHERPA KITCHEN,2670751.0,Restaurant,Risk 1 (High),2701 W LAWRENCE AVE,CHICAGO,IL,60625.0,...,Pass w/ Conditions,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,41.968442,-87.696304,"{'longitude': '41.968442053562335', 'latitude'...",NaN,NaN,NaN,NaN,NaN
16,2320311,MAXWELL STREET EXPRESS,MAXWELL STREET EXPRESS,1647725.0,Restaurant,Risk 2 (Medium),1058 W 87TH ST,CHICAGO,IL,60620.0,...,Fail,5. PROCEDURES FOR RESPONDING TO VOMITING AND D...,41.736080,-87.650859,"{'longitude': '41.7360797687383', 'latitude': ...",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194783,60244,ZARAGOZA RESTAURANT,ZARAGOZA RESTAURANT,1904141.0,Restaurant,Risk 1 (High),4852 S PULASKI RD,CHICAGO,IL,60632.0,...,Pass,31. CLEAN MULTI-USE UTENSILS AND SINGLE SERVIC...,41.804468,-87.723633,"{'longitude': '41.80446825925762', 'latitude':...",NaN,NaN,NaN,NaN,NaN
194796,67743,EL MILAGRO,EL MILAGRO,43519.0,Restaurant,Risk 1 (High),3050-3052 W 26TH ST,CHICAGO,IL,60623.0,...,Pass,32. FOOD AND NON-FOOD CONTACT SURFACES PROPERL...,41.844687,-87.702188,"{'longitude': '41.844686619118384', 'latitude'...",NaN,NaN,NaN,NaN,NaN
194803,154219,SEDGWICK'S,SEDGWICK'S,30486.0,Restaurant,Risk 1 (High),1935 N SEDGWICK ST,CHICAGO,IL,60614.0,...,Pass,"30. FOOD IN ORIGINAL CONTAINER, PROPERLY LABEL...",41.917559,-87.638645,"{'longitude': '41.91755928707187', 'latitude':...",NaN,NaN,NaN,NaN,NaN
194805,67759,FINLEY MAHONY'S,FINLEY MAHONY'S,1801271.0,Restaurant,Risk 1 (High),3701 N BROADWAY,CHICAGO,IL,60613.0,...,Pass w/ Conditions,3. POTENTIALLY HAZARDOUS FOOD MEETS TEMPERATUR...,41.949250,-87.648260,"{'longitude': '41.94924954780584', 'latitude':...",NaN,NaN,NaN,NaN,NaN


In [14]:
%%capture
!pip install folium

In [15]:
# Import necessary packages 
from folium import folium, plugins
from IPython.display import HTML

%matplotlib inline

In [16]:
# Generate map
m = folium.Map([41.8600, -87.6298], zoom_start=10)

# Convert to (n, 2) nd-array format for heatmap
inspections_arr = inspections.sample(20000)[["Latitude", "Longitude"]].values

# Plot heatmap
m.add_child(plugins.HeatMap(inspections_arr.tolist(), radius=10))

# High Risk Facilities

In [18]:
high_risk = inspections.loc[data['Risk'] == ('Risk 1 (High)')]

In [19]:
# Generate map
m1 = folium.Map([41.8600, -87.6298], zoom_start=10)

# Convert to (n, 2) nd-array format for heatmap
inspections_arr = high_risk.sample(20000)[["Latitude", "Longitude"]].values

# Plot heatmap
m1.add_child(plugins.HeatMap(inspections_arr.tolist(), radius=10))

# Medium Risk

In [22]:
medium_risk = inspections.loc[data['Risk'] == ('Risk 2 (Medium)')]
# Generate map
m2 = folium.Map([41.8600, -87.6298], zoom_start=10)

# Convert to (n, 2) nd-array format for heatmap
inspections_arr = medium_risk[["Latitude", "Longitude"]].values

# Plot heatmap
m2.add_child(plugins.HeatMap(inspections_arr.tolist(), radius=10))

# Low Risk

In [23]:
low_risk = inspections.loc[data['Risk'] == ('Risk 3 (Low)')]
# Generate map
m2 = folium.Map([41.8600, -87.6298], zoom_start=10)

# Convert to (n, 2) nd-array format for heatmap
inspections_arr = low_risk[["Latitude", "Longitude"]].values

# Plot heatmap
m2.add_child(plugins.HeatMap(inspections_arr.tolist(), radius=10))